In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [6]:
# use large scale here to get the output really close to the value in dataset, which is important to Mode.
# large scale doesn't blowup circuit size in Mode, so fine.
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.mode(x)
# IMPORTANT: Here, we use traditional Mode, so there is no result_error at all, since Mode pick the result from dataset, instead of calculating the result
# But we still have error stuffs here to just conform to computation_to_model function, though the error's not used at all.
error = 0
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:190: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = torch.tensor(mode_within(fil_x_1d, 0))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:171: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:200: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  resul

In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 8, param_scale: 8, scale_input_multiplier: 10) ------------->

+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error  | max_error | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00078201294 | -0.0015640259 | 0         | -0.0015640259 | 0.00078201294  | 0.0015640259     | 0.0015640259  | 0             | 0.0000012230885    | -0.000010077487    | 0.000010077487         |
+----------------+---------------+-----------+---------------+----------------+--------

==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":302,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709638558103}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.3694641590118408 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.6015625
==== Generating Proof ====
proof:  {'instances': [['c2b6968dfce66ece687a00d15dea969a5dab54b503c9ac9d0ad01308cedbb32e', '0100000000000000000000000000000000000000000000000000000000000000', '9a4d000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x09005856f36a0334abc86dde4da7d4c7c2292203ffcf82c6c7f1ca6d4162e75b2f60fd629c9c402d7b0a377c13f5092827c8269c931973f47587b0dbfe209a4c0313116dabdceddb76197d862b867834de61201d94a2a82c499f79b37001421f1f6e48f6426c41ff8bc28ce95c0b7a56f6d3c0fc75cce976521238276f2a96e92ea7e60438d2b59b781d1130a8601e1f6135b8083da0c8a65833b87c17f7e6bc03fddb04d41294895d72348f43d9e9820225195a3165252fb1ce5aea84c7ef5d2ddb9c2c0c40d173d2c5bd24624a19be7c57b5241152a18b6b400dbef04182f3173a8de204840c6dacdfc1f297a09f5af07daae2a6f3986cce2791701b9a2e3512bb5780d3f60d1fa11e5749780074811079f109ab8c6483f5af56c84914ec1603fd18bc5

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [77.6015625]
